# Calculating nearest mrt station to bus stop

In [ ]:
# bus stops geodataframe
bus_stops_gdf = geopandas.GeoDataFrame(bus_stops, 
                                       geometry = geopandas.points_from_xy(bus_stops.Longitude, bus_stops.Latitude),
                                       crs = "EPSG:4326")
bus_stops_gdf

In [ ]:
# bus routes geodataframe
bus_routes_gdf = geopandas.GeoDataFrame(bus_route_names, 
                                       geometry = geopandas.points_from_xy(bus_route_names.Longitude, bus_route_names.Latitude),
                                       crs = "EPSG:4326")
bus_routes_gdf

In [ ]:

# train station exits geodataframe
train_station_exits_gdf = geopandas.read_file("./TrainStationExit/Train_Station_Exit_Layer.shp")
train_station_exits_gdf

In [ ]:
bus_routes_gdf_3857 = bus_routes_gdf.to_crs(3857)
train_station_exits_gdf_3857 = train_station_exits_gdf.to_crs(3857)

nearest_train_station_output = []
for i, row in tqdm(bus_routes_gdf_3857.iterrows()):

    # Compute the distance from each bus stop to all train station exits
    distances = train_station_exits_gdf_3857.distance(row['geometry'])

    nearest_train_station_output.append({
        'Bus Service': row['ServiceNo'],
        'Route Direction': row['Direction'],
        'Bus Stop Code': row['BusStopCode'],
        'Description': row['Description'],
        'Nearest Train Station': train_station_exits_gdf_3857.loc[int(distances.argmin()), 'stn_name'],
        'Distance': float(distances.min())
    })

nearest_train_stations = pd.DataFrame(nearest_train_station_output)
nearest_train_stations

In [ ]:
bus_stops_with_mrt = nearest_train_stations[nearest_train_stations['Distance'] < 150]
number_of_mrt = bus_stops_with_mrt.groupby(['Bus Service', 'Route Direction']).size().reset_index(name = 'Count')
number_of_mrt.sort_values('Count', ascending = False)

## Bus Routes Data

In [73]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

C:\Users\kaili\AppData\Local\Temp\ipykernel_24756\1680479668.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()


,ServiceNo,Direction,geometry
0,10,1,LINESTRING (11570925.355747303 150749.03122670...
1,10,2,LINESTRING (11551610.561061133 144087.88490860...
2,100,1,LINESTRING (11562943.630025776 150347.12028417...
3,100,2,LINESTRING (11553774.516701924 145960.08849871...
4,100A,1,LINESTRING (11562943.630025776 150347.12028417...
...,...,...,...
719,992,1,LINESTRING (11549515.381030776 150294.50067879...
720,992,2,LINESTRING (11547835.02338783 151709.373424986...
721,993,1,LINESTRING (11548357.891036088 148329.89223418...
722,9A,1,LINESTRING (11569327.609406518 147460.11332459...


## MRT Stations Data

In [74]:
# mrt stations geodataframe
import os
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

mrt_stations_gdf = geopandas.read_file('./TrainStation_Jul2024/RapidTransitSystemStation.shp')
mrt_stations_gdf_3857 = mrt_stations_gdf.to_crs(3857)
mrt_stations_gdf_3857

,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
0,0,None,None,MRT,GALI BATU DEPOT,"POLYGON ((11549880.181 155286.262, 11549893.09..."
1,0,None,None,MRT,HILLVIEW MRT STATION,"POLYGON ((11551320.421 151685.479, 11551324.86..."
2,0,None,None,MRT,BEAUTY WORLD MRT STATION,"POLYGON ((11552265.127 149269.279, 11552255.26..."
3,0,None,None,MRT,HUME MRT STATION,"POLYGON ((11551478.151 150855.083, 11551485.53..."
4,0,None,None,MRT,BUKIT PANJANG MRT STATION,"POLYGON ((11550666.118 153603.709, 11550698.62..."
...,...,...,...,...,...,...
225,0,None,None,MRT,KATONG PARK MRT STATION,"POLYGON ((11564432.509 144548.028, 11564447.75..."
226,0,None,None,MRT,MARINE TERRACE MRT STATION,"POLYGON ((11567908.523 145472.086, 11567908.42..."
227,0,None,None,MRT,TANJONG RHU MRT STATION,"POLYGON ((11563110.672 144334.178, 11563108.03..."
228,0,None,None,MRT,MARINE PARADE MRT STATION,"POLYGON ((11566600.443 144955.407, 11566582.69..."


In [76]:
mrt_stations_gdf2 = geopandas.read_file('./mrt_stations/mrt_stations.shp')

mrt_stations_gdf2["lat_lng"] = geopandas.points_from_xy(mrt_stations_gdf2["LATITUDE"], mrt_stations_gdf2["LONGITUDE"])
mrt_stations_gdf2["line"] = mrt_stations_gdf2["CODE"].str.slice(0, 2) 

mrt_lines_code = ["NS", "EW", "CC", "NE", "TE", "DT", "CG"]

mrt_gdf_wo_lrt = mrt_stations_gdf2[mrt_stations_gdf2["line"].isin(mrt_lines_code)].reset_index()
mrt_gdf_wo_lrt = mrt_gdf_wo_lrt.drop("index", axis = 1)

mrt_gdf_wo_lrt["STN_NAM_DE"] = mrt_gdf_wo_lrt["BUILDING"].str.split('(').str[0].str.strip()

mrt_gdf_wo_lrt2 = mrt_gdf_wo_lrt[["CODE", "STATION_NA", "STN_NAM_DE", "LINE", "line", "COLOR", "geometry"]]

mrt_gdf_wo_lrt2["Order"] = pd.to_numeric(mrt_gdf_wo_lrt2["CODE"].str.slice(2, None))

In [77]:
new_brown_line = ['TANJONG RHU MRT STATION','KATONG PARK MRT STATION', 'TANJONG KATONG MRT STATION', 
                  'MARINE PARADE MRT STATION', 'MARINE TERRACE MRT STATION', 'SIGLAP MRT STATION', 'BAYSHORE MRT STATION']

In [78]:
n = len(new_brown_line)
add_tel_stations = []
for i in range(n):
    no = i + 23
    station_name = new_brown_line[i]
    add_tel_stations.append({
        "CODE": "TE" + str(no),
        "STATION_NA": station_name,
        "STN_NAM_DE": station_name,
        "LINE": "Thomson-East Coast Line",
        "line": "TE",
        "COLOR": "Brown",
        "Order": no
    })

new_tel_stations = pd.DataFrame(add_tel_stations)

mrt_gdf_wo_lrt2 = pd.concat([mrt_gdf_wo_lrt2, new_tel_stations], axis = 0)
mrt_gdf_wo_lrt2

,CODE,STATION_NA,STN_NAM_DE,LINE,line,COLOR,geometry,Order
0,NS1,Jurong East,JURONG EAST MRT STATION,North-South Line,NS,Red,POINT (17869.057 35038.969),1
1,NS2,Bukit Batok,BUKIT BATOK MRT STATION,North-South Line,NS,Red,POINT (18679.322 36794.926),2
2,NS3,Bukit Gombak,BUKIT GOMBAK MRT STATION,North-South Line,NS,Red,POINT (18926.913 37854.036),3
3,NS4,Choa Chu Kang,CHOA CHU KANG MRT STATION,North-South Line,NS,Red,POINT (18101.247 40812.122),4
4,NS5,Yew Tee,YEW TEE MRT STATION,North-South Line,NS,Red,POINT (18438.983 42158.018),5
...,...,...,...,...,...,...,...,...
2,TE25,TANJONG KATONG MRT STATION,TANJONG KATONG MRT STATION,Thomson-East Coast Line,TE,Brown,None,25
3,TE26,MARINE PARADE MRT STATION,MARINE PARADE MRT STATION,Thomson-East Coast Line,TE,Brown,None,26
4,TE27,MARINE TERRACE MRT STATION,MARINE TERRACE MRT STATION,Thomson-East Coast Line,TE,Brown,None,27
5,TE28,SIGLAP MRT STATION,SIGLAP MRT STATION,Thomson-East Coast Line,TE,Brown,None,28


In [79]:
mrt_stations_3857_2 = mrt_gdf_wo_lrt2.merge(mrt_stations_gdf_3857, how = "left", on = "STN_NAM_DE")

mrt_stations_3857_2

,CODE,STATION_NA,STN_NAM_DE,LINE,line,COLOR,geometry_x,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry_y
0,NS1,Jurong East,JURONG EAST MRT STATION,North-South Line,NS,Red,POINT (17869.057 35038.969),1,0,None,None,MRT,"POLYGON ((11548513.722 148484.605, 11548515.37..."
1,NS2,Bukit Batok,BUKIT BATOK MRT STATION,North-South Line,NS,Red,POINT (18679.322 36794.926),2,0,None,None,MRT,"POLYGON ((11549318.598 150099.924, 11549295.94..."
2,NS3,Bukit Gombak,BUKIT GOMBAK MRT STATION,North-South Line,NS,Red,POINT (18926.913 37854.036),3,0,None,None,MRT,"POLYGON ((11549629.35 151190.998, 11549628.967..."
3,NS4,Choa Chu Kang,CHOA CHU KANG MRT STATION,North-South Line,NS,Red,POINT (18101.247 40812.122),4,0,None,None,MRT,"POLYGON ((11548794.664 154207.851, 11548779.42..."
4,NS5,Yew Tee,YEW TEE MRT STATION,North-South Line,NS,Red,POINT (18438.983 42158.018),5,0,None,None,MRT,"POLYGON ((11549092.655 155476.939, 11549088.92..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,TE26,MARINE PARADE MRT STATION,MARINE PARADE MRT STATION,Thomson-East Coast Line,TE,Brown,None,26,0,None,None,MRT,"POLYGON ((11566600.443 144955.407, 11566582.69..."
219,TE27,MARINE TERRACE MRT STATION,MARINE TERRACE MRT STATION,Thomson-East Coast Line,TE,Brown,None,27,0,None,None,MRT,"POLYGON ((11567908.523 145472.086, 11567908.42..."
220,TE28,SIGLAP MRT STATION,SIGLAP MRT STATION,Thomson-East Coast Line,TE,Brown,None,28,0,None,None,MRT,"POLYGON ((11569421.424 145813.022, 11569353.34..."
221,TE29,BAYSHORE MRT STATION,BAYSHORE MRT STATION,Thomson-East Coast Line,TE,Brown,None,29,0,None,None,MRT,"POLYGON ((11571007.301 146270.809, 11570994.34..."


### Thomson-East Coast Line

In [80]:
#brown_line = ['WOODLANDS NORTH MRT STATION',
            #   'WOODLANDS MRT STATION',
            #   'WOODLANDS SOUTH MRT STATION',
            #   'SPRINGLEAF MRT STATION',
            #   'LENTOR MRT STATION',
            #   'MAYFLOWER MRT STATION',
            #   'BRIGHT HILL MRT STATION',
            #   'UPPER THOMSON MRT STATION',
            #   'CALDECOTT MRT STATION',
            #   'STEVENS MRT STATION',
            #   'NAPIER MRT STATION',
            #   'ORCHARD BOULEVARD MRT STATION',
            #   'ORCHARD MRT STATION',
            #   'GREAT WORLD MRT STATION',
            #   'HAVELOCK MRT STATION',
            #   'OUTRAM PARK MRT STATION',
            #   'MAXWELL MRT STATION',
            #   'SHENTON WAY MRT STATION',
            #   'MARINA BAY MRT STATION',
            #   'GARDENS BY THE BAY MRT STATION',
            #   'TANJONG RHU MRT STATION',
            #   'KATONG PARK MRT STATION',
            #   'TANJONG KATONG MRT STATION',
            #   'MARINE PARADE MRT STATION',
            #   'MARINE TERRACE MRT STATION',
            #   'SIGLAP MRT STATION',
            #   'BAYSHORE MRT STATION',
            #   'BEDOK SOUTH MRT STATION',
            #   'SUNGEI BEDOK MRT STATION']

# # filter brown line stations from mrt stations geodataframe
# brown_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(brown_line)]

# # order brown line stations based on list
# brown_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(brown_line_stations_gdf['STN_NAM_DE'],
#                                                        categories = brown_line,
#                                                        ordered = True)
# brown_line_stations_gdf = brown_line_stations_gdf.sort_values('STN_NAM_DE')
# brown_line_stations_gdf

TEL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "TE"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
TEL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "TE"]

TEL_stations_gdf["order"] = pd.to_numeric(TEL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
TEL_stations_gdf = TEL_stations_gdf.sort_values('order')
TEL_stations_gdf




,CODE,STATION_NA,STN_NAM_DE,LINE,line,COLOR,geometry_x,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry_y,order
185,TE1,Woodlands North,WOODLANDS NORTH MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (22699.991 47770.395),1,0,None,None,MRT,"POLYGON ((11553290.504 161340.499, 11553308.78...",1
186,TE2,Woodlands,WOODLANDS MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (22949.919 46417.522),2,0,None,None,MRT,"POLYGON ((11553305.733 159892.909, 11553302.44...",2
187,TE2,Woodlands,WOODLANDS MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (22949.919 46417.522),2,0,None,None,MRT,"POLYGON ((11553655.571 159844.445, 11553714.07...",2
188,TE3,Woodlands South,WOODLANDS SOUTH MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (23542.555 45459.759),3,0,None,None,MRT,"POLYGON ((11554340.403 158819.168, 11554344.10...",3
189,TE4,Springleaf,SPRINGLEAF MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (26279.328 42162.934),4,0,None,None,MRT,"POLYGON ((11556983.209 155700.488, 11556972.59...",4
190,TE5,Lentor,LENTOR MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (28269.897 40827.794),5,0,None,None,MRT,"POLYGON ((11559127.808 154037.405, 11559127.68...",5
191,TE6,Mayflower,MAYFLOWER MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (28361.595 39274.947),6,0,None,None,MRT,"POLYGON ((11559099.098 152920.462, 11559094.58...",6
192,TE7,Bright Hill,BRIGHT HILL MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (27952.707 38370.444),7,0,None,None,MRT,"POLYGON ((11558731.827 151811.306, 11558686.82...",7
193,TE8,Upper Thomson,UPPER THOMSON MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (27953.247 37390.004),8,0,None,None,MRT,"POLYGON ((11558480.214 150916.092, 11558483.67...",8
194,TE9,Caldecott,CALDECOTT MRT STATION,Thomson-East Coast Line,TE,Brown,POINT (28761.576 35491.343),9,0,None,CC17_CDT STN.zip,MRT,"POLYGON ((11559344.392 148875.759, 11559342.18...",9


#### Route Visualisation

In [81]:
import folium
import geopandas
from shapely.geometry import mapping
from shapely.geometry import LineString

# ordered list of station coordinates
TEL_stations_gdf['coordinates'] = TEL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
TEL_stations_gdf

# create a linestring for the  route
TEL_route = LineString(TEL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
TEL_route_gdf = geopandas.GeoDataFrame(geometry = [TEL_route], crs = "EPSG:3857")
TEL_route_gdf_4326 = TEL_route_gdf.to_crs(4326)

# create base map
TEL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add TEL route layer
folium.GeoJson(
    data = mapping(TEL_route_gdf_4326),
    name = 'TEL',
    style_function = lambda x: {
        'color': '#9D5B25',
        'weight': 4
    }
).add_to(TEL_map)

TEL_map


In [82]:
# calculate the overlapping distance of each bus route and the TEL
buffer_distance = 150
buffered_TEL = TEL_route.buffer(buffer_distance)

overlap_distance_output_TEL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_TEL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_TEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Thomson-East Coast Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_TEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Thomson-East Coast Line',
            'Intersection': overlap
            })

overlap_distance_TEL = geopandas.GeoDataFrame(overlap_distance_output_TEL)

# filter bus routes with overlap
bus_routes_overlap_TEL = overlap_distance_TEL[overlap_distance_TEL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_TEL

724it [00:00, 1935.39it/s]


,Bus Service,Direction,Overlap Distance,MRT line,Intersection
349,36,1,13593.103519,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
351,36B,1,11870.846806,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
392,47,1,6834.465678,Thomson-East Coast Line,MULTILINESTRING ((11569869.707853401 146101.91...
225,196e,1,6707.001743,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
350,36A,1,6238.955197,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
...,...,...,...,...,...
184,178A,1,126.008576,Thomson-East Coast Line,LINESTRING (11553397.54702328 159976.868225107...
183,178,2,125.297974,Thomson-East Coast Line,LINESTRING (11553272.25125933 159976.123976987...
553,851e,2,106.919252,Thomson-East Coast Line,LINESTRING (11559513.575879322 142672.92639385...
619,911A,1,105.055072,Thomson-East Coast Line,LINESTRING (11553375.39444461 159966.066837988...


In [83]:
bus_routes_overlap = bus_routes_overlap_TEL
bus_routes_overlap

,Bus Service,Direction,Overlap Distance,MRT line,Intersection
349,36,1,13593.103519,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
351,36B,1,11870.846806,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
392,47,1,6834.465678,Thomson-East Coast Line,MULTILINESTRING ((11569869.707853401 146101.91...
225,196e,1,6707.001743,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
350,36A,1,6238.955197,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
...,...,...,...,...,...
184,178A,1,126.008576,Thomson-East Coast Line,LINESTRING (11553397.54702328 159976.868225107...
183,178,2,125.297974,Thomson-East Coast Line,LINESTRING (11553272.25125933 159976.123976987...
553,851e,2,106.919252,Thomson-East Coast Line,LINESTRING (11559513.575879322 142672.92639385...
619,911A,1,105.055072,Thomson-East Coast Line,LINESTRING (11553375.39444461 159966.066837988...


### Downtown Line 

In [84]:
# blue_line = ['SUNGEI KADUT MRT STATION',
#              'BUKIT PANJANG MRT STATION',
#              'CASHEW MRT STATION',
#              'HILLVIEW MRT STATION',
#              'HUME MRT STATION',
#              'BEAUTY WORLD MRT STATION',
#              'KING ALBERT PARK MRT STATION',
#              'SIXTH AVENUE MRT STATION',
#              'TAN KAH KEE MRT STATION',
#              'BOTANIC GARDENS MRT STATION',
#              'STEVENS MRT STATION',
#              'NEWTON MRT STATION',
#              'LITTLE INDIA MRT STATION',
#              'ROCHOR MRT STATION',
#              'BUGIS MRT STATION',
#              'PROMENADE MRT STATION',
#              'BAYFRONT MRT STATION',
#              'DOWNTOWN MRT STATION',
#              'TELOK AYER MRT STATION',
#              'CHINATOWN MRT STATION',
#              'FORT CANNING MRT STATION',
#              'BENCOOLEN MRT STATION',
#              'JALAN BESAR MRT STATION',
#              'BENDEMEER MRT STATION',
#              'GEYLANG BAHRU MRT STATION',
#              'MATTAR MRT STATION',
#              'MACPHERSON MRT STATION',
#              'UBI MRT STATION',
#              'KAKI BUKIT MRT STATION',
#              'BEDOK NORTH MRT STATION',
#              'BEDOK RESERVOIR MRT STATION',
#              'TAMPINES WEST MRT STATION',
#              'TAMPINES MRT STATION',
#              'TAMPINES EAST MRT STATION',
#              'UPPER CHANGI MRT STATION',
#              'EXPO MRT STATION']

# # filter blue line stations from mrt stations geodataframe
# blue_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(blue_line)]

# # order blue line stations based on list
# blue_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(blue_line_stations_gdf['STN_NAM_DE'],
#                                                        categories = blue_line,
#                                                        ordered = True)
# blue_line_stations_gdf = blue_line_stations_gdf.sort_values('STN_NAM_DE')
# blue_line_stations_gdf
DTL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "DT"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
DTL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "DT"]

DTL_stations_gdf["order"] = pd.to_numeric(DTL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
DTL_stations_gdf = DTL_stations_gdf.sort_values('order')
DTL_stations_gdf


,CODE,STATION_NA,STN_NAM_DE,LINE,line,COLOR,geometry_x,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry_y,order
141,DT1,Bukit Panjang,BUKIT PANJANG MRT STATION,Downtown Line,DT,Blue,POINT (20011.398 40108.684),1,0,None,None,MRT,"POLYGON ((11550666.118 153603.709, 11550698.62...",1
142,DT2,Cashew,CASHEW MRT STATION,Downtown Line,DT,Blue,POINT (20334.559 39092.859),2,0,None,None,MRT,"POLYGON ((11551010.357 152545.896, 11551024.66...",2
143,DT3,Hillview,HILLVIEW MRT STATION,Downtown Line,DT,Blue,POINT (20666.068 38266.79),3,0,None,None,MRT,"POLYGON ((11551320.421 151685.479, 11551324.86...",3
144,DT5,Beauty World,BEAUTY WORLD MRT STATION,Downtown Line,DT,Blue,POINT (21598.166 35931.236),5,0,None,None,MRT,"POLYGON ((11552265.127 149269.279, 11552255.26...",5
145,DT6,King Albert Park,KING ALBERT PARK MRT STATION,Downtown Line,DT,Blue,POINT (22444.49 35316.248),6,0,None,None,MRT,"POLYGON ((11553177.948 148708.332, 11553217.08...",6
146,DT7,Sixth Avenue,SIXTH AVENUE MRT STATION,Downtown Line,DT,Blue,POINT (23947.749 34785.023),7,0,None,None,MRT,"POLYGON ((11554594.564 148204.756, 11554598.90...",7
147,DT8,Tan Kah Kee,TAN KAH KEE MRT STATION,Downtown Line,DT,Blue,POINT (25155.622 34198.805),8,0,None,None,MRT,"POLYGON ((11555673.915 147692.277, 11555708.09...",8
148,DT9,Botanic Gardens,BOTANIC GARDENS MRT STATION,Downtown Line,DT,Blue,POINT (26074.172 33859.535),9,0,None,CC19_BTN STN.zip,MRT,"POLYGON ((11556592.183 147107.79, 11556577.023...",9
149,DT9,Botanic Gardens,BOTANIC GARDENS MRT STATION,Downtown Line,DT,Blue,POINT (26074.172 33859.535),9,0,None,None,MRT,"POLYGON ((11556730.423 147258.342, 11556757.97...",9
151,DT10,Stevens,STEVENS MRT STATION,Downtown Line,DT,Blue,POINT (27188.23 33591.661),10,0,None,None,MRT,"POLYGON ((11557783.878 146982.248, 11557809.15...",10


#### Route Visualisation

In [85]:
# ordered list of  station coordinates
DTL_stations_gdf['coordinates'] = DTL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
DTL_stations_gdf

# create a linestring for the route
DTL_route = LineString(DTL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
DTL_route_gdf = geopandas.GeoDataFrame(geometry = [DTL_route], crs = "EPSG:3857")
DTL_route_gdf_4326 = DTL_route_gdf.to_crs(4326)

# create base map
DTL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add DTL route layer
folium.GeoJson(
    data = mapping(DTL_route_gdf_4326),
    name = 'DTL',
    style_function = lambda x: {
        'color': '#005ec4',
        'weight': 4
    }
).add_to(DTL_map)

DTL_map


In [86]:
# calculate the overlapping distance of each bus route and the blue line
buffered_DTL = DTL_route.buffer(buffer_distance)

overlap_distance_output_DTL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_DTL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_DTL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Downtown Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_DTL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Downtown Line',
            'Intersection': overlap
            })

overlap_distance_DTL = pd.DataFrame(overlap_distance_output_DTL)
overlap_distance_DTL

# filter bus routes with overlap with DTL
bus_routes_overlap_DTL = overlap_distance_DTL[overlap_distance_DTL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_DTL

724it [00:00, 1959.78it/s]


,Bus Service,Direction,Overlap Distance,MRT line,Intersection
478,67,2,14747.338450,Downtown Line,MULTILINESTRING ((11570925.355747303 150749.03...
477,67,1,13707.296228,Downtown Line,MULTILINESTRING ((11550550.736296775 153470.92...
252,23,1,12799.109602,Downtown Line,MULTILINESTRING ((11570925.355747303 150749.03...
164,170,1,12344.012344,Downtown Line,MULTILINESTRING ((11561192.006549312 145157.14...
165,170,2,12169.748634,Downtown Line,MULTILINESTRING ((11550642.203908678 153685.76...
...,...,...,...,...,...
53,123,2,182.361588,Downtown Line,LINESTRING (11559748.600892276 143847.70722962...
63,129A,1,30.953370,Downtown Line,LINESTRING (11570187.875770293 149738.22617307...
629,920,1,10.275598,Downtown Line,MULTILINESTRING ((11550876.228613876 153427.09...
630,922,1,10.275598,Downtown Line,MULTILINESTRING ((11550876.228613876 153427.09...


In [87]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_DTL], axis = 0)
bus_routes_overlap

,Bus Service,Direction,Overlap Distance,MRT line,Intersection
349,36,1,13593.103519,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
351,36B,1,11870.846806,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
392,47,1,6834.465678,Thomson-East Coast Line,MULTILINESTRING ((11569869.707853401 146101.91...
225,196e,1,6707.001743,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
350,36A,1,6238.955197,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
...,...,...,...,...,...
53,123,2,182.361588,Downtown Line,LINESTRING (11559748.600892276 143847.70722962...
63,129A,1,30.953370,Downtown Line,LINESTRING (11570187.875770293 149738.22617307...
629,920,1,10.275598,Downtown Line,MULTILINESTRING ((11550876.228613876 153427.09...
630,922,1,10.275598,Downtown Line,MULTILINESTRING ((11550876.228613876 153427.09...


### North-East Line 

In [88]:
# purple_line = [
#     'HARBOURFRONT MRT STATION',
#     'OUTRAM PARK MRT STATION',
#     'CHINATOWN MRT STATION',
#     'CLARKE QUAY MRT STATION',
#     'DHOBY GHAUT MRT STATION',
#     'LITTLE INDIA MRT STATION',
#     'FARRER PARK MRT STATION',
#     'BOON KENG MRT STATION',
#     'POTONG PASIR MRT STATION',
#     'WOODLEIGH MRT STATION',
#     'SERANGOON MRT STATION',
#     'KOVAN MRT STATION',
#     'HOUGANG MRT STATION',
#     'BUANGKOK MRT STATION',
#     'SENGKANG MRT STATION',
#     'PUNGGOL MRT STATION'
# ]


# # filter purple line stations from mrt stations geodataframe
# purple_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(purple_line)]

# # order purple line stations based on list
# purple_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(purple_line_stations_gdf['STN_NAM_DE'],
#                                                        categories = purple_line,
#                                                        ordered = True)
# purple_line_stations_gdf = purple_line_stations_gdf.sort_values('STN_NAM_DE')
# purple_line_stations_gdf

NEL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "NE"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
NEL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "NE"]

NEL_stations_gdf["order"] = pd.to_numeric(NEL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
NEL_stations_gdf = NEL_stations_gdf.sort_values('order')
NEL_stations_gdf

,CODE,STATION_NA,STN_NAM_DE,LINE,line,COLOR,geometry_x,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry_y,order
78,NE1,HarbourFront,HARBOURFRONT MRT STATION,North East Line,NE,Purple,POINT (26688.037 27545.853),1,0,None,NE1_HBF STN.zip,MRT,"POLYGON ((11557256.56 140840.359, 11557256.277...",1
79,NE1,HarbourFront,HARBOURFRONT MRT STATION,North East Line,NE,Purple,POINT (26688.037 27545.853),1,0,None,CC29_HBF STN.zip,MRT,"POLYGON ((11557408.287 140823.192, 11557327.29...",1
80,NE3,Outram Park,OUTRAM PARK MRT STATION,North East Line,NE,Purple,POINT (28681.249 29130.522),3,0,None,C109_S_OTP_MAIN_A_R19_P2.zip,MRT,"POLYGON ((11559237.753 142695.209, 11559251.83...",3
81,NE3,Outram Park,OUTRAM PARK MRT STATION,North East Line,NE,Purple,POINT (28681.249 29130.522),3,0,None,NE3_OTP STN.zip,MRT,"POLYGON ((11559350.232 142623.057, 11559353.15...",3
82,NE3,Outram Park,OUTRAM PARK MRT STATION,North East Line,NE,Purple,POINT (28681.249 29130.522),3,0,None,None,MRT,"POLYGON ((11559308.372 142682.402, 11559298.43...",3
83,NE4,Chinatown,CHINATOWN MRT STATION,North East Line,NE,Purple,POINT (29124.948 29643.48),4,0,None,NE4_CNT STN.zip,MRT,"POLYGON ((11559695.295 142864.198, 11559671.98...",4
84,NE4,Chinatown,CHINATOWN MRT STATION,North East Line,NE,Purple,POINT (29124.948 29643.48),4,0,None,None,MRT,"POLYGON ((11559751.172 143121.613, 11559752.03...",4
85,NE5,Clarke Quay,CLARKE QUAY MRT STATION,North East Line,NE,Purple,POINT (29473.125 30088.704),5,0,None,NE5_CQY STN.zip,MRT,"POLYGON ((11560054.558 143242.668, 11560050.58...",5
86,NE6,Dhoby Ghaut,DHOBY GHAUT MRT STATION,North East Line,NE,Purple,POINT (29354.408 31340.251),6,0,None,None,MRT,"POLYGON ((11559965.996 144666.945, 11560032.60...",6
87,NE6,Dhoby Ghaut,DHOBY GHAUT MRT STATION,North East Line,NE,Purple,POINT (29354.408 31340.251),6,0,None,NE6_DBG STN.zip,MRT,"POLYGON ((11560057.669 144610.631, 11560099.83...",6


#### Route Visualisation

In [89]:
# ordered list of purple line station coordinates
NEL_stations_gdf['coordinates'] = NEL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
NEL_stations_gdf

# create a linestring for the route
NEL_route = LineString(NEL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
NEL_route_gdf = geopandas.GeoDataFrame(geometry = [NEL_route], crs = "EPSG:3857")
NEL_route_gdf_4326 = NEL_route_gdf.to_crs(4326)

# create base map
NEL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add NEL route layer
folium.GeoJson(
    data = mapping(NEL_route_gdf_4326),
    name = 'NEL',
    style_function = lambda x: {
        'color': '#9900aa',
        'weight': 4
    }
).add_to(NEL_map)

NEL_map

In [90]:
# calculate the overlapping distance of each bus route and the line
buffered_NEL = NEL_route.buffer(buffer_distance)

overlap_distance_output_NEL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_NEL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_NEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-East Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_NEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-East Line',
            'Intersection': overlap
            })

overlap_distance_NEL = pd.DataFrame(overlap_distance_output_NEL)
overlap_distance_NEL

# filter bus routes with overlap with NEL line
bus_routes_overlap_NEL = overlap_distance_NEL[overlap_distance_NEL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_NEL

724it [00:00, 2740.20it/s]


,Bus Service,Direction,Overlap Distance,MRT line,Intersection
19,107M,1,11395.931068,North-East Line,MULTILINESTRING ((11565278.908793872 152589.82...
108,147,2,7852.430597,North-East Line,MULTILINESTRING ((11558613.257532349 142208.58...
5,101,1,6218.377800,North-East Line,MULTILINESTRING ((11562943.630025776 150347.12...
18,107,2,6101.333864,North-East Line,MULTILINESTRING ((11562354.096129267 147456.09...
539,82,1,5966.269041,North-East Line,MULTILINESTRING ((11566344.824867642 156275.50...
...,...,...,...,...,...
36,116A,1,100.181411,North-East Line,LINESTRING (11563104.946318492 150483.91377265...
12,105,2,73.738111,North-East Line,LINESTRING (11562884.183542915 150390.74853951...
22,109A,1,71.275668,North-East Line,LINESTRING (11562943.630025776 150347.12028417...
8,102B,1,70.896183,North-East Line,LINESTRING (11565506.87539269 155096.450562328...


In [91]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_NEL], axis = 0)
bus_routes_overlap

,Bus Service,Direction,Overlap Distance,MRT line,Intersection
349,36,1,13593.103519,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
351,36B,1,11870.846806,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
392,47,1,6834.465678,Thomson-East Coast Line,MULTILINESTRING ((11569869.707853401 146101.91...
225,196e,1,6707.001743,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
350,36A,1,6238.955197,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
...,...,...,...,...,...
36,116A,1,100.181411,North-East Line,LINESTRING (11563104.946318492 150483.91377265...
12,105,2,73.738111,North-East Line,LINESTRING (11562884.183542915 150390.74853951...
22,109A,1,71.275668,North-East Line,LINESTRING (11562943.630025776 150347.12028417...
8,102B,1,70.896183,North-East Line,LINESTRING (11565506.87539269 155096.450562328...


### North-South Line 

In [92]:
NSL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "NS"]["STATION_NA"].tolist()

# filter red line stations from mrt stations geodataframe
NSL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "NS"]

NSL_stations_gdf["order"] = pd.to_numeric(NSL_stations_gdf["CODE"].str.slice(2, None))

# order red line stations based on list
NSL_stations_gdf = NSL_stations_gdf.sort_values('order')
NSL_stations_gdf

,CODE,STATION_NA,STN_NAM_DE,LINE,line,COLOR,geometry_x,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry_y,order
0,NS1,Jurong East,JURONG EAST MRT STATION,North-South Line,NS,Red,POINT (17869.057 35038.969),1,0,None,None,MRT,"POLYGON ((11548513.722 148484.605, 11548515.37...",1
1,NS2,Bukit Batok,BUKIT BATOK MRT STATION,North-South Line,NS,Red,POINT (18679.322 36794.926),2,0,None,None,MRT,"POLYGON ((11549318.598 150099.924, 11549295.94...",2
2,NS3,Bukit Gombak,BUKIT GOMBAK MRT STATION,North-South Line,NS,Red,POINT (18926.913 37854.036),3,0,None,None,MRT,"POLYGON ((11549629.35 151190.998, 11549628.967...",3
3,NS4,Choa Chu Kang,CHOA CHU KANG MRT STATION,North-South Line,NS,Red,POINT (18101.247 40812.122),4,0,None,None,MRT,"POLYGON ((11548794.664 154207.851, 11548779.42...",4
4,NS5,Yew Tee,YEW TEE MRT STATION,North-South Line,NS,Red,POINT (18438.983 42158.018),5,0,None,None,MRT,"POLYGON ((11549092.655 155476.939, 11549088.92...",5
5,NS7,Kranji,KRANJI MRT STATION,North-South Line,NS,Red,POINT (20078.587 45204.517),7,0,None,None,MRT,"POLYGON ((11550658.471 158600.225, 11550639.63...",7
6,NS8,Marsiling,MARSILING MRT STATION,North-South Line,NS,Red,POINT (21407.041 46026.513),8,0,None,None,MRT,"POLYGON ((11551976.264 159472.321, 11551970.71...",8
7,NS9,Woodlands,WOODLANDS MRT STATION,North-South Line,NS,Red,POINT (22741.598 46501.786),9,0,None,None,MRT,"POLYGON ((11553305.733 159892.909, 11553302.44...",9
8,NS9,Woodlands,WOODLANDS MRT STATION,North-South Line,NS,Red,POINT (22741.598 46501.786),9,0,None,None,MRT,"POLYGON ((11553655.571 159844.445, 11553714.07...",9
9,NS10,Admiralty,ADMIRALTY MRT STATION,North-South Line,NS,Red,POINT (24402.387 46918.507),10,0,None,None,MRT,"POLYGON ((11554977.061 160321.073, 11554965.95...",10


#### Route Visualisation

In [93]:
# ordered list of red line station coordinates
NSL_stations_gdf['coordinates'] = NSL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
NSL_stations_gdf


# create a linestring for the red line route
NSL_route = LineString(NSL_stations_gdf['coordinates'])
NSL_route

# convert from LineString to geodataframe
NSL_route_gdf = geopandas.GeoDataFrame(geometry = [NSL_route], crs = "EPSG:3857")
NSL_route_gdf_4326 = NSL_route_gdf.to_crs(4326)

# create base map
NSL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add NSL route layer
folium.GeoJson(
    data = mapping(NSL_route_gdf_4326),
    name = 'NEL',
    style_function = lambda x: {
        'color': '#d42e12',
        'weight': 4
    }
).add_to(NSL_map)

NSL_map

In [94]:
buffered_NSL = NSL_route.buffer(buffer_distance)

overlap_distance_output_NSL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_NSL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_NSL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-South Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_NSL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-South Line',
            'Intersection': overlap
            })

overlap_distance_NSL = pd.DataFrame(overlap_distance_output_NSL)
overlap_distance_NSL

bus_routes_overlap_NSL = overlap_distance_NSL[overlap_distance_NSL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_NSL

724it [00:00, 2015.98it/s]


,Bus Service,Direction,Overlap Distance,MRT line,Intersection
666,965,1,10583.682845,North-South Line,MULTILINESTRING ((11553436.063567095 160044.46...
572,858,1,6501.666038,North-South Line,MULTILINESTRING ((11553397.54702328 159976.868...
682,972M,1,6287.758118,North-South Line,MULTILINESTRING ((11559199.100152956 146381.00...
152,167,2,6209.606483,North-South Line,MULTILINESTRING ((11560578.466324601 142830.45...
552,851e,1,5726.875630,North-South Line,MULTILINESTRING ((11558981.499660706 159025.23...
...,...,...,...,...,...
88,138B,1,46.077493,North-South Line,LINESTRING (11560369.616077418 152487.46172084...
292,269A,1,41.274189,North-South Line,LINESTRING (11560369.616077418 152487.46172084...
181,177,1,23.956965,North-South Line,MULTILINESTRING ((11549515.381030776 150294.50...
602,88B,1,20.204034,North-South Line,LINESTRING (11560613.114441711 152463.15596228...


In [95]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_NSL], axis = 0)
bus_routes_overlap

,Bus Service,Direction,Overlap Distance,MRT line,Intersection
349,36,1,13593.103519,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
351,36B,1,11870.846806,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
392,47,1,6834.465678,Thomson-East Coast Line,MULTILINESTRING ((11569869.707853401 146101.91...
225,196e,1,6707.001743,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
350,36A,1,6238.955197,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
...,...,...,...,...,...
88,138B,1,46.077493,North-South Line,LINESTRING (11560369.616077418 152487.46172084...
292,269A,1,41.274189,North-South Line,LINESTRING (11560369.616077418 152487.46172084...
181,177,1,23.956965,North-South Line,MULTILINESTRING ((11549515.381030776 150294.50...
602,88B,1,20.204034,North-South Line,LINESTRING (11560613.114441711 152463.15596228...


### East-West Line 

In [96]:
EWL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "EW"]["STATION_NA"].tolist()
EWL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "EW"]

EWL_stations_gdf["order"] = pd.to_numeric(EWL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
EWL_stations_gdf = EWL_stations_gdf.sort_values('order')
EWL_stations_gdf

,CODE,STATION_NA,STN_NAM_DE,LINE,line,COLOR,geometry_x,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry_y,order
36,EW1,Pasir Ris,PASIR RIS MRT STATION,East-West Line,EW,Green,POINT (40905.597 39449.982),1,0,None,None,MRT,"POLYGON ((11571575.189 152769.789, 11571541.17...",1
37,EW2,Tampines,TAMPINES MRT STATION,East-West Line,EW,Green,POINT (40444.973 37267.026),2,0,None,None,MRT,"POLYGON ((11571164.394 150577.16, 11571131.418...",2
38,EW2,Tampines,TAMPINES MRT STATION,East-West Line,EW,Green,POINT (40444.973 37267.026),2,0,None,None,MRT,"POLYGON ((11570846.566 150813.451, 11570821.24...",2
39,EW3,Simei,SIMEI MRT STATION,East-West Line,EW,Green,POINT (41361.228 36149.753),3,0,None,C306_S_SIM_MAIN_A_R19_P3.zip,MRT,"POLYGON ((11572075.163 149463.256, 11572025.14...",3
40,EW4,Tanah Merah,TANAH MERAH MRT STATION,East-West Line,EW,Green,POINT (40579.1 34379.432),4,0,None,C305_S_TNM_MAIN_A_R19_P3.zip,MRT,"POLYGON ((11571200.167 147713.186, 11571193.69...",4
41,EW5,Bedok,BEDOK MRT STATION,East-West Line,EW,Green,POINT (38757.952 34024.705),5,0,None,None,MRT,"POLYGON ((11569380.786 147376.24, 11569377.809...",5
42,EW6,Kembangan,KEMBANGAN MRT STATION,East-West Line,EW,Green,POINT (36861.961 33699.344),6,0,None,None,MRT,"POLYGON ((11567458.866 147038.723, 11567453.12...",6
43,EW7,Eunos,EUNOS MRT STATION,East-West Line,EW,Green,POINT (35779.988 33560.586),7,0,None,None,MRT,"POLYGON ((11566378.754 146889.55, 11566372.968...",7
45,EW8,Paya Lebar,PAYA LEBAR MRT STATION,East-West Line,EW,Green,POINT (34648.724 33375.669),8,0,None,None,MRT,"POLYGON ((11565245.034 146717.237, 11565242.68...",8
44,EW8,Paya Lebar,PAYA LEBAR MRT STATION,East-West Line,EW,Green,POINT (34648.724 33375.669),8,0,None,CC9_PYL STN.zip,MRT,"POLYGON ((11565169.458 146753.277, 11565203.22...",8


#### Route Visualisation

In [97]:
EWL_stations_gdf = geopandas.GeoDataFrame(EWL_stations_gdf, geometry='geometry_y')

EWL_stations_gdf['coordinates'] = EWL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
EWL_stations_gdf

# create a linestring for the route
EWL_route = LineString(EWL_stations_gdf['coordinates'])
EWL_route

# convert from LineString to geodataframe
EWL_route_gdf = geopandas.GeoDataFrame(geometry = [EWL_route], crs = "EPSG:3857")
EWL_route_gdf_4326 = EWL_route_gdf.to_crs(4326)

# create base map
EWL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add EWL route layer
folium.GeoJson(
    data = mapping(EWL_route_gdf_4326),
    name = 'NEL',
    style_function = lambda x: {
        'color': '#009645',
        'weight': 4
    }
).add_to(EWL_map)

EWL_map

In [98]:
buffered_EWL = EWL_route.buffer(buffer_distance)

overlap_distance_output_EWL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_EWL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_EWL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'East-West Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_EWL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'East-West Line',
            'Intersection': overlap
            })

overlap_distance_EWL = pd.DataFrame(overlap_distance_output_EWL)
overlap_distance_EWL

bus_routes_overlap_EWL = overlap_distance_EWL[overlap_distance_EWL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_EWL

724it [00:00, 1845.22it/s]


,Bus Service,Direction,Overlap Distance,MRT line,Intersection
442,63,1,12987.276479,East-West Line,MULTILINESTRING ((11566296.397064393 146880.92...
234,2,2,12440.274381,East-West Line,MULTILINESTRING ((11559228.30061612 142395.858...
258,24,1,10469.713735,East-West Line,MULTILINESTRING ((11565207.157161014 146843.03...
607,9,1,8631.436216,East-West Line,MULTILINESTRING ((11569327.609406518 147460.11...
406,51,2,8626.680077,East-West Line,MULTILINESTRING ((11548347.742894938 148397.07...
...,...,...,...,...,...
369,39A,1,79.801168,East-West Line,LINESTRING (11571490.526500894 152774.11363990...
590,87,2,79.410554,East-West Line,LINESTRING (11569327.609406518 147460.11332459...
187,17A,1,79.410554,East-West Line,LINESTRING (11569327.609406518 147460.11332459...
589,87,1,76.194445,East-West Line,LINESTRING (11569287.456862345 147524.86953427...


In [99]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_EWL], axis = 0)
bus_routes_overlap

,Bus Service,Direction,Overlap Distance,MRT line,Intersection
349,36,1,13593.103519,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
351,36B,1,11870.846806,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
392,47,1,6834.465678,Thomson-East Coast Line,MULTILINESTRING ((11569869.707853401 146101.91...
225,196e,1,6707.001743,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
350,36A,1,6238.955197,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
...,...,...,...,...,...
369,39A,1,79.801168,East-West Line,LINESTRING (11571490.526500894 152774.11363990...
590,87,2,79.410554,East-West Line,LINESTRING (11569327.609406518 147460.11332459...
187,17A,1,79.410554,East-West Line,LINESTRING (11569327.609406518 147460.11332459...
589,87,1,76.194445,East-West Line,LINESTRING (11569287.456862345 147524.86953427...


### Circle Line

In [100]:
CCL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "CC"]["STATION_NA"].tolist()

CCL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "CC"]

CCL_stations_gdf["order"] = pd.to_numeric(CCL_stations_gdf["CODE"].str.slice(2, None))

# order circle stations based on list
CCL_stations_gdf = CCL_stations_gdf.sort_values('order')
CCL_stations_gdf

,CODE,STATION_NA,STN_NAM_DE,LINE,line,COLOR,geometry_x,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry_y,order
102,CC1,Dhoby Ghaut,DHOBY GHAUT MRT STATION,Circle Line,CC,Orange,POINT (29443.882 31252.572),1,0,None,None,MRT,"POLYGON ((11559965.996 144666.945, 11560032.60...",1
103,CC1,Dhoby Ghaut,DHOBY GHAUT MRT STATION,Circle Line,CC,Orange,POINT (29443.882 31252.572),1,0,None,NE6_DBG STN.zip,MRT,"POLYGON ((11560057.669 144610.631, 11560099.83...",1
104,CC1,Dhoby Ghaut,DHOBY GHAUT MRT STATION,Circle Line,CC,Orange,POINT (29443.882 31252.572),1,0,None,None,MRT,"POLYGON ((11560057.669 144610.631, 11560048.94...",1
105,CC2,Bras Basah,BRAS BASAH MRT STATION,Circle Line,CC,Orange,POINT (29930.767 31025.902),2,0,None,CC2_BBS STN.zip,MRT,"POLYGON ((11560546.586 144457.107, 11560562.03...",2
106,CC3,Esplanade,ESPLANADE MRT STATION,Circle Line,CC,Orange,POINT (30421.993 30671.628),3,0,None,CC3_EPN STN.zip,MRT,"POLYGON ((11561238.875 143970.789, 11561242.22...",3
107,CC4,Promenade,PROMENADE MRT STATION,Circle Line,CC,Orange,POINT (31008.362 30709.258),4,0,None,CC4_PMN STN.zip,MRT,"POLYGON ((11561749.024 144127.673, 11561774.92...",4
108,CC4,Promenade,PROMENADE MRT STATION,Circle Line,CC,Orange,POINT (31008.362 30709.258),4,0,None,None,MRT,"POLYGON ((11561725.013 144035.618, 11561732.57...",4
109,CC5,Nicoll Highway,NICOLL HIGHWAY MRT STATION,Circle Line,CC,Orange,POINT (31374.165 31347.153),5,0,None,CC5_NCH STN.zip,MRT,"POLYGON ((11561957.863 144678.764, 11561987.01...",5
110,CC6,Stadium,STADIUM MRT STATION,Circle Line,CC,Orange,POINT (32676.348 31683.941),6,0,None,CC6_SDM STN.zip,MRT,"POLYGON ((11563397.464 145082.341, 11563398.51...",6
111,CC7,Mountbatten,MOUNTBATTEN MRT STATION,Circle Line,CC,Orange,POINT (33476.563 32058.742),7,0,None,CC7_MBT STN.zip\r\nCC7_MBT STN.zip,MRT,"POLYGON ((11564045.348 145402.245, 11564050.87...",7


#### Route Visualisation

In [101]:
CCL_stations_gdf = geopandas.GeoDataFrame(CCL_stations_gdf, geometry='geometry_y')

CCL_stations_gdf['coordinates'] =CCL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
CCL_stations_gdf

# create a linestring for the circle line route
CCL_route = LineString(CCL_stations_gdf['coordinates'])
CCL_route

# convert from LineString to geodataframe
CCL_route_gdf = geopandas.GeoDataFrame(geometry = [CCL_route], crs = "EPSG:3857")
CCL_route_gdf_4326 = CCL_route_gdf.to_crs(4326)

# create base map
CCL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add CCL route layer
folium.GeoJson(
    data = mapping(CCL_route_gdf_4326),
    name = 'CCL',
    style_function = lambda x: {
        'color': '#fa9e0d',
        'weight': 4
    }
).add_to(CCL_map)

CCL_map

In [102]:
buffered_CCL = CCL_route.buffer(buffer_distance)

overlap_distance_output_CCL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_CCL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_CCL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Circle Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_CCL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Circle Line',
            'Intersection': overlap
            })

overlap_distance_CCL = pd.DataFrame(overlap_distance_output_CCL)
overlap_distance_CCL

bus_routes_overlap_CCL = overlap_distance_CCL[overlap_distance_CCL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_CCL

724it [00:00, 2556.52it/s]


,Bus Service,Direction,Overlap Distance,MRT line,Intersection
497,70M,1,8631.266710,Circle Line,MULTILINESTRING ((11563104.94635849 150483.913...
129,158,1,7683.686214,Circle Line,MULTILINESTRING ((11562943.630025776 150347.12...
236,200,1,7571.348025,Circle Line,MULTILINESTRING ((11554113.023137262 145748.82...
0,10,1,6990.730262,Circle Line,MULTILINESTRING ((11565002.69374407 145678.918...
1,10,2,6780.148840,Circle Line,MULTILINESTRING ((11552847.31276766 143033.072...
...,...,...,...,...,...
92,13A,1,139.165513,Circle Line,LINESTRING (11560269.529441975 150530.19989975...
142,162,1,102.368728,Circle Line,LINESTRING (11559261.958614238 150279.92370331...
36,116A,1,100.181166,Circle Line,LINESTRING (11563104.94635849 150483.913531073...
496,70B,1,100.181166,Circle Line,LINESTRING (11563104.94635849 150483.913531073...


In [104]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_CCL], axis = 0)
bus_routes_overlap

,Bus Service,Direction,Overlap Distance,MRT line,Intersection
349,36,1,13593.103519,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
351,36B,1,11870.846806,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
392,47,1,6834.465678,Thomson-East Coast Line,MULTILINESTRING ((11569869.707853401 146101.91...
225,196e,1,6707.001743,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
350,36A,1,6238.955197,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...
...,...,...,...,...,...
92,13A,1,139.165513,Circle Line,LINESTRING (11560269.529441975 150530.19989975...
142,162,1,102.368728,Circle Line,LINESTRING (11559261.958614238 150279.92370331...
36,116A,1,100.181166,Circle Line,LINESTRING (11563104.94635849 150483.913531073...
496,70B,1,100.181166,Circle Line,LINESTRING (11563104.94635849 150483.913531073...


# Bus Stops in Overlap

In [105]:
# find the bus stops that belong to each intersection between the bus routes and the MRT lines
bus_routes_overlap_gdf_3857 = geopandas.GeoDataFrame(bus_routes_overlap,
                                                     geometry = bus_routes_overlap['Intersection'],
                                                     crs = "EPSG:3857")

overlap_count_output = []

for i, row in tqdm(bus_routes_overlap_gdf_3857.iterrows()):
    bus_no = row['Bus Service']
    direction = row['Direction']
    bus = bus_routes_gdf_3857[(bus_routes_gdf_3857['ServiceNo'] == bus_no) & (bus_routes_gdf_3857['Direction'] == direction)]
    for j, stop in bus.iterrows():
        if row['Intersection'].contains(stop.geometry):
            overlap_count_output.append({
                'Bus Service': bus_no,
                'Direction': direction,
                'Bus Stop Code': stop['BusStopCode'],
                'Description': stop['Description'],
                'MRT line': row['MRT line'],
                'geometry': stop.geometry
            })

overlap_bus_stops = geopandas.GeoDataFrame(overlap_count_output, crs = "EPSG:3857")


2735it [00:37, 73.43it/s]


In [106]:
# get the number of bus stops in the intersection between each bus route mrt line
overlap_count = overlap_bus_stops.groupby(['Bus Service', 'Direction', 'MRT line']).size().reset_index(name = 'Count').sort_values('Count', ascending = False)

bus_routes_overlap = bus_routes_overlap.merge(overlap_count, on = ['Bus Service', 'Direction', 'MRT line'], how = 'outer')
bus_routes_overlap = bus_routes_overlap[bus_routes_overlap['Count'].notna()]
bus_routes_overlap['Count'] = bus_routes_overlap['Count'].astype(int)

In [107]:
# top 10 routes with significant overlap
bus_routes_overlap.sort_values(['Overlap Distance'], ascending = False).head(10)

,Bus Service,Direction,Overlap Distance,MRT line,Intersection,Count
1894,67,2,14747.338450,Downtown Line,MULTILINESTRING ((11570925.355747303 150749.03...,40
1887,67,1,13707.296228,Downtown Line,MULTILINESTRING ((11550550.736296775 153470.92...,34
1281,36,1,13593.103519,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728104 145953.86...,24
1655,63,1,12987.276479,East-West Line,MULTILINESTRING ((11566296.397064393 146880.92...,38
1048,23,1,12799.109602,Downtown Line,MULTILINESTRING ((11570925.355747303 150749.03...,13
997,2,2,12440.274381,East-West Line,MULTILINESTRING ((11559228.30061612 142395.858...,29
737,170,1,12344.012344,Downtown Line,MULTILINESTRING ((11561192.006549312 145157.14...,33
743,170,2,12169.748634,Downtown Line,MULTILINESTRING ((11550642.203908678 153685.76...,29
1295,36B,1,11870.846806,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...,22
97,107M,1,11395.931068,North-East Line,MULTILINESTRING ((11565278.908793872 152589.82...,33
